In [1]:
import os
import shutil
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import kagglehub
import random
import tensorflow
import keras

In [2]:
fruit_path = kagglehub.dataset_download("raghavrpotdar/fresh-and-stale-images-of-fruits-and-vegetables")

food_path = kagglehub.dataset_download("anthonytherrien/image-classification-32-classes-food")

In [3]:
%ls "$fruit_path"

 Volume in drive C has no label.
 Volume Serial Number is D2D6-E9FB

 Directory of C:\Users\PC\.cache\kagglehub\datasets\raghavrpotdar\fresh-and-stale-images-of-fruits-and-vegetables\versions\1

29-05-2025  22:10    <DIR>          .
29-05-2025  22:10    <DIR>          ..
29-05-2025  22:10    <DIR>          fresh_apple
29-05-2025  22:10    <DIR>          fresh_banana
29-05-2025  22:10    <DIR>          fresh_bitter_gourd
29-05-2025  22:10    <DIR>          fresh_capsicum
29-05-2025  22:10    <DIR>          fresh_orange
29-05-2025  22:10    <DIR>          fresh_tomato
29-05-2025  22:10               180 ImageLabels.txt
29-05-2025  22:10    <DIR>          stale_apple
29-05-2025  22:10    <DIR>          stale_banana
29-05-2025  22:10    <DIR>          stale_bitter_gourd
29-05-2025  22:10    <DIR>          stale_capsicum
29-05-2025  22:10    <DIR>          stale_orange
29-05-2025  22:10    <DIR>          stale_tomato
               1 File(s)            180 bytes
              14 Dir(s)  133

In [4]:
%ls "$food_path"

 Volume in drive C has no label.
 Volume Serial Number is D2D6-E9FB

 Directory of C:\Users\PC\.cache\kagglehub\datasets\anthonytherrien\image-classification-32-classes-food\versions\1

30-05-2025  11:16    <DIR>          .
30-05-2025  11:16    <DIR>          ..
30-05-2025  11:17    <DIR>          image
               0 File(s)              0 bytes
               3 Dir(s)  133,106,348,032 bytes free


In [5]:
print("Files in the dataset directory:")
print(os.listdir(r'C:\Users\PC\.cache\kagglehub\datasets\anthonytherrien\image-classification-32-classes-food\versions\1\image'))

Files in the dataset directory:
['alfredo', 'broccoli', 'brownie', 'cake', 'carrot', 'cereal', 'cheese', 'chicken', 'chocolate', 'coffee', 'cookie', 'corn', 'couscous', 'crab', 'donut', 'egg', 'fajitas', 'fries', 'grilledcheese', 'hotdog', 'icecream', 'macncheese', 'nachos', 'nuggets', 'rice', 'salad', 'salmon', 'shrimp', 'soup', 'steak', 'sushi', 'tartare']


In [6]:
food_path = r"C:\Users\PC\.cache\kagglehub\datasets\anthonytherrien\image-classification-32-classes-food\versions\1\image"

In [7]:
subset_dir = "visionXXXX"

os.makedirs(os.path.join(subset_dir), exist_ok=True)

train_dir = os.path.join(subset_dir, "train")
val_dir = os.path.join(subset_dir, "val")


os.makedirs(os.path.join(train_dir, "apple"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "banana"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "orange"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "tomato"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "capsicum"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "bitter gourd"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "broccoli"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "carrot"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "rice"), exist_ok=True)

os.makedirs(os.path.join(val_dir, "apple"), exist_ok=True)
os.makedirs(os.path.join(val_dir, "banana"), exist_ok=True)
os.makedirs(os.path.join(val_dir, "orange"), exist_ok=True)
os.makedirs(os.path.join(val_dir, "tomato"), exist_ok=True)
os.makedirs(os.path.join(val_dir, "capsicum"), exist_ok=True)
os.makedirs(os.path.join(val_dir, "bitter gourd"), exist_ok=True)
os.makedirs(os.path.join(val_dir, "broccoli"), exist_ok=True)
os.makedirs(os.path.join(val_dir, "carrot"), exist_ok=True)
os.makedirs(os.path.join(val_dir, "rice"), exist_ok=True)

orange_path = os.path.join(fruit_path, r"C:\Users\PC\.cache\kagglehub\datasets\raghavrpotdar\fresh-and-stale-images-of-fruits-and-vegetables\versions\1", "fresh_orange")
banana_path = os.path.join(fruit_path, r"C:\Users\PC\.cache\kagglehub\datasets\raghavrpotdar\fresh-and-stale-images-of-fruits-and-vegetables\versions\1", "fresh_banana")
apple_path = os.path.join(fruit_path, r"C:\Users\PC\.cache\kagglehub\datasets\raghavrpotdar\fresh-and-stale-images-of-fruits-and-vegetables\versions\1", "fresh_apple")
cap_path = os.path.join(fruit_path, r"C:\Users\PC\.cache\kagglehub\datasets\raghavrpotdar\fresh-and-stale-images-of-fruits-and-vegetables\versions\1", "fresh_capsicum")
tom_path = os.path.join(fruit_path, r"C:\Users\PC\.cache\kagglehub\datasets\raghavrpotdar\fresh-and-stale-images-of-fruits-and-vegetables\versions\1", "fresh_tomato")
bit_path = os.path.join(fruit_path, r"C:\Users\PC\.cache\kagglehub\datasets\raghavrpotdar\fresh-and-stale-images-of-fruits-and-vegetables\versions\1", "fresh_bitter_gourd")
broc_path = os.path.join(food_path, r'C:\Users\PC\.cache\kagglehub\datasets\anthonytherrien\image-classification-32-classes-food\versions\1\image', "broccoli")
car_path = os.path.join(food_path, r'C:\Users\PC\.cache\kagglehub\datasets\anthonytherrien\image-classification-32-classes-food\versions\1\image', "carrot")
rice_path = os.path.join(food_path, r'C:\Users\PC\.cache\kagglehub\datasets\anthonytherrien\image-classification-32-classes-food\versions\1\image', "rice")

In [8]:
orange_img = sorted([f for f in os.listdir(orange_path) if f.endswith(".png")])
banana_img = sorted([f for f in os.listdir(banana_path) if f.endswith(".png")])
apple_img = sorted([f for f in os.listdir(apple_path) if f.endswith(".png")])
tom_img = sorted([f for f in os.listdir(tom_path) if f.endswith(".jpg")])
cap_img = sorted([f for f in os.listdir(cap_path) if f.endswith(".jpg")])
bit_img = sorted([f for f in os.listdir(bit_path) if f.endswith(".jpg")])
broc_img = sorted([f for f in os.listdir(broc_path) if f.endswith(".png")])
car_img = sorted([f for f in os.listdir(car_path) if f.endswith(".png")])
rice_img = sorted([f for f in os.listdir(rice_path) if f.endswith(".png")])

for img in orange_img[:500]:
    shutil.copy(os.path.join(orange_path, img), os.path.join(train_dir, "orange"))
for img in orange_img[-250:]:
    shutil.copy(os.path.join(orange_path, img), os.path.join(val_dir, "orange"))

for img in banana_img[:500]:
    shutil.copy(os.path.join(banana_path, img), os.path.join(train_dir, "banana"))
for img in banana_img[-250:]:
    shutil.copy(os.path.join(banana_path, img), os.path.join(val_dir, "banana"))

for img in apple_img[:500]:
    shutil.copy(os.path.join(apple_path, img), os.path.join(train_dir, "apple"))
for img in apple_img[-250:]:
    shutil.copy(os.path.join(apple_path, img), os.path.join(val_dir, "apple"))

for img in tom_img[:500]:
    shutil.copy(os.path.join(tom_path, img), os.path.join(train_dir, "tomato"))
for img in tom_img[-250:]:
    shutil.copy(os.path.join(tom_path, img), os.path.join(val_dir, "tomato"))

for img in cap_img[:500]:
    shutil.copy(os.path.join(cap_path, img), os.path.join(train_dir, "capsicum"))
for img in cap_img[-250:]:
    shutil.copy(os.path.join(cap_path, img), os.path.join(val_dir, "capsicum"))

for img in bit_img[:200]:
    shutil.copy(os.path.join(bit_path, img), os.path.join(train_dir, "bitter gourd"))
for img in bit_img[-100:]:
    shutil.copy(os.path.join(bit_path, img), os.path.join(val_dir, "bitter gourd"))

for img in broc_img[:175]:
    shutil.copy(os.path.join(broc_path, img), os.path.join(train_dir, "broccoli"))
for img in broc_img[-88:]:
    shutil.copy(os.path.join(broc_path, img), os.path.join(val_dir, "broccoli"))

for img in car_img[:180]:
    shutil.copy(os.path.join(car_path, img), os.path.join(train_dir, "carrot"))
for img in car_img[-108:]:
    shutil.copy(os.path.join(car_path, img), os.path.join(val_dir, "carrot"))

for img in rice_img[:180]:
    shutil.copy(os.path.join(rice_path, img), os.path.join(train_dir, "rice"))
for img in rice_img[-104:]:
    shutil.copy(os.path.join(rice_path, img), os.path.join(val_dir, "rice"))

In [9]:
print("Created a train/val subset in:", subset_dir)

Created a train/val subset in: visionXXXX


In [10]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomRotation(60),
    transforms.RandomHorizontalFlip(p=0.8),
    transforms.RandomVerticalFlip(p=0.6),  # Try vertical flip
    transforms.RandomAffine(degrees=40, translate=(0.6, 0.6)),  # Slight translations
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # More aggressive cropping
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
val_transform = transforms.Compose ([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [11]:
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(val_dir, transform=val_transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [23]:
class_names = train_dataset.classes
print("Classes:", class_names)

Classes: ['apple', 'banana', 'bitter gourd', 'broccoli', 'capsicum', 'carrot', 'orange', 'rice', 'tomato']


In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device, "as device")

Using cuda as device


In [25]:
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [26]:
model = models.mobilenet_v2(pretrained=True)

num_features = model.classifier[1].in_features
# Modify the last layer for 5 classes
model.classifier = nn.Sequential(
    nn.Linear(num_features, 9),
    nn.Dropout(0.4)
)

In [27]:
class_weights = torch.tensor([1.0, 1.0, 1.0,1.0,1.0,1.0,1.0,1.0,1.0]).to(device)  
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=1e-6, weight_decay = 0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, verbose=True)

In [28]:
model.to(device)

from tqdm import tqdm


def train(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total = 0

        # Training Phase
        for inputs, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels).item()
            total += labels.size(0)

        # Calculate training loss and accuracy
        train_loss = running_loss / total
        train_acc = running_corrects / total

        # Validation Phase
        model.eval()
        val_running_loss = 0.0
        val_running_corrects = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f"Validating Epoch {epoch+1}/{num_epochs}", leave=False):
                inputs, labels = inputs.to(device), labels.to(device)

                # Forward pass
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                # Metrics calculation
                _, preds = torch.max(outputs, 1)
                val_running_loss += loss.item() * inputs.size(0)
                val_running_corrects += torch.sum(preds == labels).item()
                val_total += labels.size(0)

        # Calculate validation loss and accuracy
        val_loss = val_running_loss / val_total
        val_acc = val_running_corrects / val_total

        # Logging the results
        print(f"Epoch {epoch+1}/{num_epochs} - "
              f"Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.3f}, "
              f"Val Loss: {val_loss:.3f}, Val Acc: {val_acc:.3f}")

        # Update the scheduler
        scheduler.step(val_loss)
        # Check for early stopping
 
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=50)

Epoch 1/50 - Train Loss: 2.123, Train Acc: 0.190, Val Loss: 2.061, Val Acc: 0.205


Epoch 2/50 - Train Loss: 1.999, Train Acc: 0.271, Val Loss: 1.903, Val Acc: 0.328


Epoch 3/50 - Train Loss: 1.860, Train Acc: 0.368, Val Loss: 1.773, Val Acc: 0.481


Epoch 4/50 - Train Loss: 1.761, Train Acc: 0.427, Val Loss: 1.635, Val Acc: 0.655


Epoch 5/50 - Train Loss: 1.673, Train Acc: 0.464, Val Loss: 1.516, Val Acc: 0.736


...

Epoch 10/50 - Train Loss: 1.288, Train Acc: 0.578, Val Loss: 0.990, Val Acc: 0.944


Epoch 11/50 - Train Loss: 1.245, Train Acc: 0.591, Val Loss: 0.908, Val Acc: 0.955


Epoch 12/50 - Train Loss: 1.210, Train Acc: 0.583, Val Loss: 0.844, Val Acc: 0.970


Epoch 13/50 - Train Loss: 1.183, Train Acc: 0.580, Val Loss: 0.777, Val Acc: 0.976


KeyboardInterrupt: 

In [29]:
torch.save(model.state_dict(), 'new_fruit_weights.pth')